### Modelo de Machine Learning Supervisado

En este último Notebook usaré **XGBoost** para predecir los productos que reordenarán los usuarios creando un modelo de predicción supervisado. para ello he **definido unas funciones** que me ayudarán a agrupar las acciones que ejecutaré durante este notebook.

En primer lugar cargamos algunad de las librerías que resultan utiles para cargar los ficheros:

* gc

* time

* numpy

* pandas

* sklearn


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Este entorno de Python 3 viene con muchas bibliotecas de análisis útiles instaladas
# Por ejemplo, aquí hay varios paquetes útiles para cargar
import gc
import time
import numpy as np # Algebre lineal
import pandas as pd # procesamiento de datos, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

# Los archivos de datos de entrada están disponibles en el directorio "DATOS".

### Funcion para cargar los datos
Mediante la función read_csv de Pandas iré cargando las distintas tablas

Para la generación de un modelo más eficiente necesito controlar el tipo de los datos por eso voy a usar la librería numpy para ello.

In [0]:
def load_data(path_data):

    #--------------------------------order_product--------------------------------
    # Ordenes ya conocidas
    priors = pd.read_csv(path_data + 'order_products__prior.csv', 
                     dtype={
                            'order_id': np.int32,
                            'product_id': np.uint16,
                            'add_to_cart_order': np.int16,
                            'reordered': np.int8})
    # Ordenes de entrenamiento
    train = pd.read_csv(path_data + 'order_products__train.csv', 
                    dtype={
                            'order_id': np.int32,
                            'product_id': np.uint16,
                            'add_to_cart_order': np.int16,
                            'reordered': np.int8})
    
    #--------------------------------orden----------------- ---------------
    # Este archivo nos dice a qué conjunto (conocido, train, test) pertenece una orden
    # Único en order_id
    # order_id en train, prior
    # Este es el orden #order_number de este usuario
    
    orders = pd.read_csv(path_data + 'orders.csv', 
                         dtype={
                                'order_id': np.int32,
                                'user_id': np.int64,
                                'eval_set': 'category',
                                'order_number': np.int16,
                                'order_dow': np.int8,
                                'order_hour_of_day': np.int8,
                                'days_since_prior_order': np.float32})

    
    #--------------------------------product--------------------------------
    
    products = pd.read_csv(path_data + 'products.csv')
    aisles = pd.read_csv(path_data + "aisles.csv")
    departments = pd.read_csv(path_data + "departments.csv")
    sample_submission = pd.read_csv(path_data + "sample_submission.csv")
    
    return priors, train, orders, products, aisles, departments, sample_submission



#### Clase tick_tock

La usaré en las siguientes funciones para agregar los datos.

In [0]:
class tick_tock:
    def __init__(self, process_name, verbose=1):
        self.process_name = process_name
        self.verbose = verbose
    def __enter__(self):
        if self.verbose:
            print(self.process_name + " begin ......")
            self.begin_time = time.time()
    def __exit__(self, type, value, traceback):
        if self.verbose:
            end_time = time.time()
            print(self.process_name + " end ......")
            print('time lapsing {0} s \n'.format(end_time - self.begin_time))
            


### Funcion 1 Vs N

**Parametros de entrada**: 

* df = DataFrame de Pandas (priors_orders_detail)

* group_columns_list = Lista de columnas que desea agrupar, podrían ser múltiples columnas (User_ID)

* agg_dict = diccionario de python

**Devuelve**:

Nuevo marco de datos de pandas con columnas originales y nuevas columnas agregadas. 
(df_new)

**Ejemplo:**


       {real_column_name: {Nombre_de_las_nuevas_columnas : method}}
       agg_dict = {'user_id':{'prod_tot_cnts':'count'},
                   'reordered':{'reorder_tot_cnts_of_this_prod':'sum'},
                   'user_buy_product_times': {'prod_order_once':lambda x: sum(x==1),
                                              'prod_order_more_than_once':lambda x: sum(x==2)}}
       ka_add_stats_features_1_vs_n(train, ['product_id'], agg_dict)

###### Proceso:
Crear columnas estadísticas, agrupar por [N columnas] y calcular estadísticas en [columna N]

* 1º Agrupa por usuario el Data Frame
* 2º Agg es un alias para Aggregate --> Por lo que agregamos lo que pasemos en agg_dict
* 3º Elimino la cabecera
* 4º Reindexo
* 5º Por ultimo uno las tablas con un merge
* 6º La funcion devuelve el ultimo dataframe

In [0]:
def ka_add_groupby_features_1_vs_n(df, group_columns_list, agg_dict, only_new_feature=True):
    
    with tick_tock("add stats features"):
        try:
            if type(group_columns_list) == list:
                pass
            else:
                raise TypeError(k + "should be a list")
        except TypeError as e:
            print(e)
            raise

        df_new = df.copy()
        grouped = df_new.groupby(group_columns_list)

        the_stats = grouped.agg(agg_dict)
        the_stats.columns = the_stats.columns.droplevel(0)
        the_stats.reset_index(inplace=True)
        if only_new_feature:
            df_new = the_stats
        else:
            df_new = pd.merge(left=df_new, right=the_stats, on=group_columns_list, how='left')

    return df_new



### Funcion N Vs 1

**Parametros de entrada:** 

* df = marco de datos pandas Matriz de características (priors_orders_detail)

* group_columns_list = Lista de columnas que desea agrupar, podría ser varias columnas (User_ID)

* target_columns_list = columna que desea calcular las estadísticas, debe ser una lista con un solo elemento

* methods_list = Métodos que desea utilizar, todos los métodos compatibles con groupby en Pandas

* agg_dict

**Devuelve:**
Nuevo marco de datos de pandas con columnas originales y nuevas columnas agregadas.
(df_new)

**Ejemplo:**

 ka_add_stats_features_n_vs_1 (train, group_columns_list = ['x0'], target_columns_list = ['x10'])

###### Proceso:



In [0]:
def ka_add_groupby_features_n_vs_1(df, group_columns_list, target_columns_list, methods_list, keep_only_stats=True, verbose=1):
#Aquí hago uso de la clase tick_tock definida previamente
    with tick_tock("add stats features", verbose):
        dicts = {"group_columns_list": group_columns_list , "target_columns_list": target_columns_list, "methods_list" :methods_list}

        for k, v in dicts.items():
            try:
                if type(v) == list:
                    pass
                else:
                    raise TypeError(k + "should be a list")
            except TypeError as e:
                print(e)
                raise

        grouped_name = ''.join(group_columns_list)
        target_name = ''.join(target_columns_list)
        combine_name = [[grouped_name] + [method_name] + [target_name] for method_name in methods_list]

        df_new = df.copy()
        grouped = df_new.groupby(group_columns_list)

        the_stats = grouped[target_name].agg(methods_list).reset_index()
        the_stats.columns = [grouped_name] + \
                            ['_%s_%s_by_%s' % (grouped_name, method_name, target_name) \
                             for (grouped_name, method_name, target_name) in combine_name]
        if keep_only_stats:
            return the_stats
        else:
            df_new = pd.merge(left=df_new, right=the_stats, on=group_columns_list, how='left')
        return df_new

**Cargo los datos parametrizados** para la cual he utilizado la funcion previamente definida

In [0]:
path_data = 'DATOS/'
priors, train, orders, products, aisles, departments, sample_submission = load_data(path_data)

# Estudio de los productos

Para el estudio de los productos voy a **crear nuevas variables**, para identificarlas las he puesto un "_" al principio de la variable

#### GRUPO DE VARIABLES 1

* prod_buy_second_time_total_cnt : Productos que se han re-comprado alguna vez
* prod_buy_first_time_total_cnt: Productos que se han comprado alguna vez
* prod_tot_cnts: Usuarios distintos que han comprado algún producto
* prod_reorder_prob: % de productos que han sido reordenados frente a los que no
* prod_reorder_ratio: % de productos que han sido reordenados del total de pedidos
* prod_reorder_times: % de veces que los productos han sido reordenados del total de pedidos

In [0]:
# Información de productos ----------------------------------------------------------------
# Agregar información del pedido a los priors establecidos
priors_orders_detail = orders.merge(right=priors, how='inner', on='order_id')

# Creacion de nuevas variables
priors_orders_detail.loc[:,'_user_buy_product_times'] = priors_orders_detail.groupby(['user_id', 'product_id']).cumcount() + 1
agg_dict = {'user_id':{'_prod_tot_cnts':'count'}, 
            'reordered':{'_prod_reorder_tot_cnts':'sum'}, 
            '_user_buy_product_times': {'_prod_buy_first_time_total_cnt':lambda x: sum(x==1),
                                        '_prod_buy_second_time_total_cnt':lambda x: sum(x==2)}}

# Uso de la función 1 Vs n para añadir las variables creadas a priors_orders_detail:
prd = ka_add_groupby_features_1_vs_n(priors_orders_detail, ['product_id'], agg_dict)

# Una vez agregadas las nuevas variables, genero unos ratios que me serán útiles para los calculos
prd['_prod_reorder_prob'] = prd._prod_buy_second_time_total_cnt / prd._prod_buy_first_time_total_cnt
prd['_prod_reorder_ratio'] = prd._prod_reorder_tot_cnts / prd._prod_tot_cnts
prd['_prod_reorder_times'] = 1 + prd._prod_reorder_tot_cnts / prd._prod_buy_first_time_total_cnt

add stats features begin ......


C:\Users\alejandro.ortega.or1\anaconda\lib\site-packages\pandas\core\groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


add stats features end ......
time lapsing 268.706999779 s 



In [0]:
prd.head()

,product_id,_prod_buy_second_time_total_cnt,_prod_buy_first_time_total_cnt,_prod_tot_cnts,_prod_reorder_tot_cnts,_prod_reorder_prob,_prod_reorder_ratio,_prod_reorder_times
0,1,276,716,1852,1136.0,0.385475,0.613391,2.586592
1,2,8,78,90,12.0,0.102564,0.133333,1.153846
2,3,36,74,277,203.0,0.486486,0.732852,3.743243
3,4,64,182,329,147.0,0.351648,0.446809,1.807692
4,5,4,6,15,9.0,0.666667,0.600000,2.500000


Ahora se puede ordenar los productos por el ratio de reordenados para ver que productos tienen más probabilidad de ser reodenados por el usuario

# Estudio de los usuarios

Para el estudio de los usuarios también voy a **crear variables que puedan resultar útiles para el modelo**


#### GRUPO DE VARIABLES 2

* user_mean_days_since_prior_order: Distancia desde la última compra (promedio)
* user_sum_days_since_prior_order: Desde el último momento de la compra, esto solo se puede calcular en la tabla de pedidos
* user_total_orders: Total de ordenes de cada uno de los usuarios

#### GRUPO DE VARIABLES 3


* user_total_products: Número total de artículos comprados por el usuario.
* user_distinct_products: El número de artículos únicos comprados por el usuario.
* user_reorder_ratio:  Número total de veces con reorder / número total de veces después del primer pedido
* user_average_basket: Numero de productos por orden de cada usuario

In [0]:

agg_dict_2 = {'order_number':{'_user_total_orders':'max'},
              'days_since_prior_order':{'_user_sum_days_since_prior_order':'sum', 
                                        '_user_mean_days_since_prior_order': 'mean'}}
# Uso de la funcion 1 Vs n para añadir las variables creadas en agg_dict_2
users = ka_add_groupby_features_1_vs_n(orders[orders.eval_set == 'prior'], ['user_id'], agg_dict_2)


agg_dict_3 = {'reordered':
              {'_user_reorder_ratio': 
               lambda x: sum(priors_orders_detail.loc[x.index,'reordered']==1)/
                         sum(priors_orders_detail.loc[x.index,'order_number'] > 1)},
              'product_id':{'_user_total_products':'count', 
                            '_user_distinct_products': lambda x: x.nunique()}}
# Uso de la funcion 1 Vs n para añadir las variables creadas en agg_dict_3
us = ka_add_groupby_features_1_vs_n(priors_orders_detail, ['user_id'], agg_dict_3)
users = users.merge(us, how='inner')

# El promedio, número mínimo y máximo de artículos por pedido
users['_user_average_basket'] = users._user_total_products / users._user_total_orders

us = orders[orders.eval_set != "prior"][['user_id', 'order_id', 'eval_set', 'days_since_prior_order']]
us.rename(index=str, columns={'days_since_prior_order': 'time_since_last_order'}, inplace=True)

users = users.merge(us, how='inner')

add stats features begin ......
add stats features end ......
time lapsing 0.37700009346 s 

add stats features begin ......
add stats features end ......
time lapsing 447.140000105 s 



In [0]:
users.head()

,user_id,_user_mean_days_since_prior_order,_user_sum_days_since_prior_order,_user_total_orders,_user_total_products,_user_distinct_products,_user_reorder_ratio,_user_average_basket,order_id,eval_set,time_since_last_order
0,1,19.555555,176.0,10,59,18,0,5.900000,1187899,train,14.0
1,2,15.230769,198.0,14,195,102,0,13.928571,1492625,train,30.0
2,3,12.090909,133.0,12,88,33,0,7.333333,2774568,test,11.0
3,4,13.750000,55.0,5,18,17,0,3.600000,329954,test,30.0
4,5,13.333333,40.0,4,37,23,0,9.250000,2196797,train,6.0


# Estudio de los datos

Debería haber muchas **variables que se pueden agregar aquí** (Eleccion de Variables).
#### GRUPO DE VARIABLES 4

* up_average_cart_position: La posición media del artículo añadido a la cesta.
* up_order_count: La cantidad de veces que el usuario compró el artículo.
* up_first_order_number: El número de pedidos que hizo el usuario para comprar el artículo por primera vez.
* up_last_order_number: El número de pedidos que el usuario compró por última vez.
* up_order_rate: Ratio de ordenes por usuario
* up_order_since_last_order: Ratio de pedido desde la ultima orden
* up_order_rate_since_first_order: Ratio de pedido desde la primera orden

In [0]:

agg_dict_4 = {'order_number':{'_up_order_count': 'count', 
                              '_up_first_order_number': 'min', 
                              '_up_last_order_number':'max'}, 
              'add_to_cart_order':{'_up_average_cart_position': 'mean'}}

# Uso de la funcion 1 Vs n para añadir las variables creadas en agg_dict_4
data = ka_add_groupby_features_1_vs_n(df=priors_orders_detail, 
                                                      group_columns_list=['user_id', 'product_id'], 
                                                      agg_dict=agg_dict_4)

data = data.merge(prd, how='inner', on='product_id').merge(users, how='inner', on='user_id')

data['_up_order_rate'] = data._up_order_count / data._user_total_orders
data['_up_order_since_last_order'] = data._user_total_orders - data._up_last_order_number
data['_up_order_rate_since_first_order'] = data._up_order_count / (data._user_total_orders - data._up_first_order_number + 1)

# añado el user_id al set de entrenamiento
train = train.merge(right=orders[['order_id', 'user_id']], how='left', on='order_id')
data = data.merge(train[['user_id', 'product_id', 'reordered']], on=['user_id', 'product_id'], how='left')

del priors_orders_detail, orders
gc.collect()

add stats features begin ......
add stats features end ......
time lapsing 24.9489998817 s 



153

In [0]:
data.head()

,user_id,product_id,_up_average_cart_position,_up_order_count,_up_first_order_number,_up_last_order_number,_prod_buy_second_time_total_cnt,_prod_buy_first_time_total_cnt,_prod_tot_cnts,_prod_reorder_tot_cnts,...,_user_distinct_products,_user_reorder_ratio,_user_average_basket,order_id,eval_set,time_since_last_order,_up_order_rate,_up_order_since_last_order,_up_order_rate_since_first_order,reordered
0,1,196,1.400000,10,1,10,4660,8000,35791,27791.0,...,18,0,5.9,1187899,train,14.0,1.0,0,1.000000,1.0
1,1,10258,3.333333,9,2,10,308,557,1946,1389.0,...,18,0,5.9,1187899,train,14.0,0.9,0,1.000000,1.0
2,1,10326,5.000000,1,5,5,1003,1923,5526,3603.0,...,18,0,5.9,1187899,train,14.0,0.1,5,0.166667,NaN
3,1,12427,3.300000,10,1,10,889,1679,6476,4797.0,...,18,0,5.9,1187899,train,14.0,1.0,0,1.000000,NaN
4,1,13032,6.333333,3,2,10,617,1286,3751,2465.0,...,18,0,5.9,1187899,train,14.0,0.3,0,0.333333,1.0


# Entrenamiento y Generación de los datos de Test

##### Xgboost

XGBoost es la abreviatura de eXtreme gradient boosting. Es una biblioteca diseñada y optimizada para algoritmos de árbol potenciados.

Su objetivo principal es presionar los límites de calculo de las máquinas para proporcionar un escalable, portátil y preciso modo **para impulsar árboles a gran escala**.

El término "**Incremento de degradado**" se propone en el artículo Acercamiento a la función codiciosa: Una máquina de refuerzo de degradado, por Friedman.

XGBoost es un nuevo tipo de algoritmo de impulso que **aprovecha el refuerzo, el diseño de hardware y las penalizaciones del modelo para crear un algoritmo de impulso muy preciso y muy rápido**. Hace que sea una alternativa viable al bosque aleatorio para su uso en aplicaciones predictivas rápidas.

**DMatrix es una estructura de datos interna que utiliza XGBoost** que está optimizada tanto para la eficiencia de la memoria como para la velocidad de entrenamiento. Puedes construir DMatrix desde numpy.arrays

#### PARAMETROS

* data (string / numpy array / scipy.sparse / pd.DataFrame / DataTable) - Fuente de datos de DMatrix. Cuando los datos son de tipo cadena, representan la ruta libsvm en formato txt, o archivo binario desde el que xgboost puede leer.
* etiqueta (lista o número 1-D array, opcional) - Etiqueta de los datos de entrenamiento.
* faltantes (flotante, opcional): valor en los datos que debe estar presente como un valor faltante. Si Ninguno, el valor predeterminado es np.nan.
* ponderación (lista o número 1-D array, opcional) - ponderación para cada instancia.
* silencioso (booleano, opcional) - Si se imprimen mensajes durante la construcción
* feature_names (lista, opcional) - Establecer nombres para las características.
* feature_types (lista, opcional) - Establecer tipos para las características.
* nthread (entero, opcional): número de subprocesos que se utilizan para cargar datos desde una matriz numpy. Si -1, utiliza el máximo de hilos disponibles en el sistema.

In [0]:
#Instalo e importo la librería xgboost: https://xgboost.readthedocs.io/en/latest/python/python_api.html
import xgboost

train = data.loc[data.eval_set == "train",:]
train.drop(['eval_set', 'user_id', 'product_id', 'order_id'], axis=1, inplace=True)
train.loc[:, 'reordered'] = train.reordered.fillna(0)

X_test = data.loc[data.eval_set == "test",:]

# Submuestra de entrenamiento
X_train, X_val, y_train, y_val = train_test_split(train.drop('reordered', axis=1), train.reordered,
                                                    test_size=0.9, random_state=42)
d_train = xgboost.DMatrix(X_train, y_train)
# Parametros que le paso a xgb
xgb_params = {
    "objective"         : "reg:logistic"
    ,"eval_metric"      : "logloss"
    ,"eta"              : 0.1
    ,"max_depth"        : 6
    ,"min_child_weight" :10
    ,"gamma"            :0.70
    ,"subsample"        :0.76
    ,"colsample_bytree" :0.95
    ,"alpha"            :2e-05
    ,"lambda"           :10
}

watchlist= [(d_train, "train")]
bst = xgboost.train(params=xgb_params, dtrain=d_train, num_boost_round=80, evals=watchlist, verbose_eval=10)
xgboost.plot_importance(bst)

C:\Users\alejandro.ortega.or1\anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\alejandro.ortega.or1\anaconda\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


[20:03:15] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 112 extra nodes, 0 pruned nodes, max_depth=6
[0]	train-logloss:0.625588
[20:03:17] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 0 pruned nodes, max_depth=6
[20:03:19] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 0 pruned nodes, max_depth=6
[20:03:21] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 0 pruned nodes, max_depth=6
[20:03:23] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 0 pruned nodes, max_depth=6
[20:03:24] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 4 pruned nodes, max_depth=6
[20:03:26] C:\Users\Administrator\Desktop\xgboost\src\tree\up

[20:04:58] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 102 extra nodes, 4 pruned nodes, max_depth=6
[20:05:01] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 96 extra nodes, 4 pruned nodes, max_depth=6
[20:05:04] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 100 extra nodes, 0 pruned nodes, max_depth=6
[20:05:07] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 0 pruned nodes, max_depth=6
[20:05:09] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 98 extra nodes, 4 pruned nodes, max_depth=6
[20:05:11] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 108 extra nodes, 4 pruned nodes, max_depth=6
[20:05:14] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruni

In [0]:
#train.head(5)
#X_test.head(5)
#watchlist
xgb_params

{'alpha': 2e-05,
 'colsample_bytree': 0.95,
 'eta': 0.1,
 'eval_metric': 'logloss',
 'gamma': 0.7,
 'lambda': 10,
 'max_depth': 6,
 'min_child_weight': 10,
 'objective': 'reg:logistic',
 'subsample': 0.76}

#### Asunciones y explicación de la predicción
Despues de realizar varias pruebas, el modelo más optimo es en el que asumo que el producto será reordenado por el usario si la predicción obtenida es superior a 0,21

In [0]:
d_test = xgboost.DMatrix(X_test.drop(['eval_set', 'user_id', 'order_id', 'reordered', 'product_id'], axis=1))
# Si la predicción d_test es superior a 0.21 asumo que es probable que el producto sea reordenado
X_test.loc[:,'reordered'] = (bst.predict(d_test) > 0.21).astype(int)
X_test.loc[:, 'product_id'] = X_test.product_id.astype(str)

# Uso la funcion N Vs 1 para añadir los valores
submit = ka_add_groupby_features_n_vs_1(X_test[X_test.reordered == 1], 
                                               group_columns_list=['order_id'],
                                               target_columns_list= ['product_id'],
                                               methods_list=[lambda x: ' '.join(set(x))], keep_only_stats=True)
submit.columns = sample_submission.columns.tolist()
submit_final = sample_submission[['order_id']].merge(submit, how='left').fillna('None')

# Por último genero el fichero de salida con lo resultados a entregar: python_test.csv
submit_final.to_csv("python_test.csv", index=False)

add stats features begin ......
add stats features end ......
time lapsing 1.80999994278 s 



In [0]:
# Tabla con los datos usados para la predicción
X_test.to_csv("tabla_prediccion.csv", index=False)

In [0]:
# Array con las predicciones de los productos
predict = bst.predict(d_test)
np.savetxt('predicciones.txt', predict, delimiter=';') 